# RAG chain

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
CHROMA_PATH = os.getenv("CHROMA_PATH")
DATA_PATH = os.getenv("DATA_PATH")

: 

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.document import Document

from langchain.document_loaders.pdf import PyPDFDirectoryLoader

from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

from langchain.vectorstores.chroma import Chroma

In [4]:
def load_documents():
    document_loader = PyPDFDirectoryLoader(DATA_PATH)
    return document_loader.load()

documents = load_documents()
print(documents[0])

page_content='Оценка AW (Academic  Withdrawal ) - выставляется в случае нарушения обучающимся правил СДУ и \nправил экзамена.  \nОценка AW равна 0 -бальному цифровому эквиваленту и рассчитывается как оценка GPA  F (Fail) \nпри среднем показатели  успеваемости. Оценка AW выставляется преподавателем -предметником \nили проектором на  основании акта, составленного решением факультета.  \nЦена AW выставляется и отпускае тся по академическим причинам. Ставится в случае нарушения \nправил экзамена. Экзаменацио нное дежурство заполняет ак т. Сдача экзамена FX по этим \nпредметам не допускается.  \nОценка "FX" - выставляется, когда итоговая оценка по дисциплине находится в пределах 25 -49. \nОбучающийся, получивший оценку ""FX"", имеет право на повторную пересдачу.  \nЧтобы сдать экзамен FX вы можете нажимать кнопку FX order на портале, где портал попросит \nденьги.  \nСдача FX стоит 1 ECTS  \nЕсли вы пропустите FX, вы повторно будете изучать данный предмет в летнем семестре.  \nЕсли портал за

In [5]:
def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 600,
        chunk_overlap = 80,
        length_function = len,
        is_separator_regex = False
    )
    return text_splitter.split_documents(documents)

In [6]:
chunks = split_documents(documents)
print(chunks[2])

page_content='преподавателя.  \nПререквизиты - это дисциплины, предшествующие изучению данной дисциплины.  \nDrop (отказ от тех или иных дисциплин с вашего расписания) дисциплин при р егистрации на \nпортале - бесплатна. На 5 -10 неделе вы платите 50% от полной стоимости предмета, на 10 -15 \nнеделе 100% сумму.  \nПо современной кредитной технологии расчет за ретейк рассчитывается по ECTS.  Летний семестр \nпо академическому календарю начинается в  начале июня. Студент учится 5 недель, экзамен сдает \nна 6 -7 неделе. Формат экзамена решается деканатом, онлайн либо оффлайн формат.' metadata={'source': 'data/faq_SDU.pdf', 'page': 0}


In [7]:
def get_embedding_function():
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    return embeddings

In [8]:
def add_to_chroma(chunks: list[Document]):
    db = Chroma(
        persist_directory = CHROMA_PATH,
        embedding_function = get_embedding_function()
    )
    db.add_documents(new_chunks, ids = new_chunks_ids)
    db.persist()

In [9]:
def get_context(question):
    persist_directory = CHROMA_PATH

    embedding = get_embedding_function()

    vectordb = Chroma.from_documents(documents=chunks,
                                     embedding=embedding,
                                     persist_directory=CHROMA_PATH)

    vectordb.persist()
    vectordb = None

    vectordb = Chroma(persist_directory=CHROMA_PATH,
                      embedding_function=embedding)

    retriever = vectordb.as_retriever()

    docs = retriever.get_relevant_documents(question)
    context_text = "\n".join(doc.page_content for doc in docs)
    return context_text

In [12]:
llm = ChatOpenAI(
                openai_api_key=openai_api_key,
                model="gpt-3.5-turbo"
)

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
CHROMA_PATH = os.getenv("CHROMA_PATH")
DATA_PATH = os.getenv("DATA_PATH")

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.document import Document

from langchain.document_loaders.pdf import PyPDFDirectoryLoader

from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

from langchain.vectorstores.chroma import Chroma

def load_documents():
    document_loader = PyPDFDirectoryLoader(DATA_PATH)
    return document_loader.load()

documents = load_documents()

def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 600,
        chunk_overlap = 80,
        length_function = len,
        is_separator_regex = False
    )
    return text_splitter.split_documents(documents)

chunks = split_documents(documents)


def get_embedding_function():
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    return embeddings

def add_to_chroma(chunks: list[Document]):
    db = Chroma(
        persist_directory = CHROMA_PATH,
        embedding_function = get_embedding_function()
    )
    db.add_documents(new_chunks, ids = new_chunks_ids)
    db.persist()

def get_context(question):
    persist_directory = CHROMA_PATH

    embedding = get_embedding_function()

    vectordb = Chroma.from_documents(documents=chunks,
                                     embedding=embedding,
                                     persist_directory=CHROMA_PATH)

    vectordb.persist()
    vectordb = None

    vectordb = Chroma(persist_directory=CHROMA_PATH,
                      embedding_function=embedding)

    retriever = vectordb.as_retriever()

    docs = retriever.get_relevant_documents(question)
    context_text = "\n".join(doc.page_content for doc in docs)
    return context_text

llm = ChatOpenAI(
                openai_api_key=openai_api_key,
                model="gpt-3.5-turbo"
)

template = """
You are a Q&A assistant for university students.
Your goal is to answer questions as accurately as possible based on context provided.
Context: {context}

---
You must answer in russian.
If you don't have information to answer the question, type "К сожалению, я не владею такой информацией. Попробуйте обратиться к эдвайзеру."
Question: {question}
"""

question= "как получить президентскую стипендию"

from langchain import PromptTemplate, LLMChain

prompt = PromptTemplate(input_variables=["context", "question"], template=template)
llm_chain = LLMChain(prompt=prompt, llm=llm)

result = llm_chain.run({"context": get_context(question), "question": question})
print(result)

In [13]:
template = """
You are a Q&A assistant for university students.
Your goal is to answer questions as accurately as possible based on context provided.
Context: {context}

---
You must answer in russian.
If you don't have information to answer the question, type "К сожалению, я не владею такой информацией. Попробуйте обратиться к эдвайзеру."
Question: {question}
"""

In [32]:
question= "как получить президентскую стипендию"

In [33]:
from langchain import PromptTemplate, LLMChain

prompt = PromptTemplate(input_variables=["context", "question"], template=template)
llm_chain = LLMChain(prompt=prompt, llm=llm)

result = llm_chain.run({"context": get_context(question), "question": question})

In [34]:
result

'Для получения президентской стипендии необходимо иметь определенные заслуги, заниматься научно-исследовательской деятельностью, быть одаренным и талантливым студентом или магистрантом. При назначении стипендии отбор осуществляется исходя из указанных критериев, при равных условиях преимущество имеют дети-сироты, дети без попечения родителей, а также дети-инвалиды.'